# Ejemplo CVA

Se calcula el CVA de un swap.

## Librerías

- Las funciones relacionadas con el modelo de Hull-White que definimos y utilizamos en los notebooks 13 y 14 han sido mejoradas y encapsuladas en el módulo `hull_white.py`.

- La función `bono_tasa_fija` se trasladó al nuevo módulo `instruments.py`.

In [11]:
from modules import instruments as inst
from modules import hull_white as hw
from modules import auxiliary as aux
import pandas as pd
import numpy as np
import math

## Auxiliares

In [30]:
frmt = {'plazo': '{:,.0f}', 'tasa': '{:.8%}', 'df': '{:.6%}', 't': '{:.4f}'}

## Data Curva

Se lee la data y se completa con el plazo en convención Act/365.

In [26]:
df_curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [27]:
df_curva['t'] = df_curva['plazo'] / 365.0

In [28]:
df_curva.tail().style.format(frmt)

,plazo,tasa,df,t
28,7306,0.92532411%,83.092401%,20.0164
29,9133,0.96571892%,78.533775%,25.0219
30,10957,0.98810273%,74.332619%,30.0192
31,14610,0.94083244%,68.619685%,40.0274
32,18262,0.84642634%,65.475678%,50.0329


## Funciones para el Modelo de HW

- `zrate`: es el cubic spline construido a partir de las columnas `t` y `tasa` de `df_curva`.
- `hwz`: es una versión más sencilla de la función `zero_hw` que sólo requiere llos parámetros `r`, `t` y `T`.
- `theta`: es la misma función que ya utilizamos.

In [5]:
gamma = 1.0
sigma = .0025

Se construyen las funciones `zrate`, `hwz` y `theta`.

In [6]:
zrate, hwz, theta = hw.get_zrate_hwzero_and_theta(
    df_curva['t'],
    df_curva['tasa'],
    gamma,
    sigma
)

Se verifica que `zrate` y más importante aún, `hwz` coincidan con los valores de la curva de mercado en los vértices de interpolación (plazos y tasas de la curva).

**Notar** el plazo utilizado para obtener el valor de `r0`.

In [43]:
r0 = zrate(.0000001)
print(f'         r0: {r0: .6%}\n')
for tup in df_curva.itertuples():
    print(f'      plazo: {tup.t:.4f}')
    print(f' tasa curva: {tup.tasa:.6%}')
    print(f'tasa spline: {zrate(tup.t):.6%}')
    print(f'   tasa hwz: {-math.log(hwz(r, 0, tup.t)) / tup.t:.6%}')
    print()

         r0:  0.078814%

      plazo: 0.0027
 tasa curva: 0.081111%
tasa spline: 0.081111%
   tasa hwz: 0.081111%

      plazo: 0.0192
 tasa curva: 0.084051%
tasa spline: 0.084051%
   tasa hwz: 0.084051%

      plazo: 0.0384
 tasa curva: 0.077967%
tasa spline: 0.077967%
   tasa hwz: 0.077967%

      plazo: 0.0575
 tasa curva: 0.077358%
tasa spline: 0.077358%
   tasa hwz: 0.077358%

      plazo: 0.0904
 tasa curva: 0.078067%
tasa spline: 0.078067%
   tasa hwz: 0.078067%

      plazo: 0.1671
 tasa curva: 0.078064%
tasa spline: 0.078064%
   tasa hwz: 0.078064%

      plazo: 0.2521
 tasa curva: 0.081103%
tasa spline: 0.081103%
   tasa hwz: 0.081103%

      plazo: 0.3425
 tasa curva: 0.078059%
tasa spline: 0.078059%
   tasa hwz: 0.078059%

      plazo: 0.4164
 tasa curva: 0.076030%
tasa spline: 0.076030%
   tasa hwz: 0.076030%

      plazo: 0.4986
 tasa curva: 0.075014%
tasa spline: 0.075014%
   tasa hwz: 0.075014%

      plazo: 0.5808
 tasa curva: 0.073593%
tasa spline: 0.073593%
   tasa h

Se verifica que la función `theta` se mantenga acotada.

In [44]:
for tup in df_curva.itertuples():
    print(f'theta({tup.t:.4f}): {theta(tup.t):.6%}')

theta(0.0027): 1.287414%
theta(0.0192): -0.943991%
theta(0.0384): 0.800401%
theta(0.0575): 0.047061%
theta(0.0904): 0.019791%
theta(0.1671): 0.337408%
theta(0.2521): -0.310565%
theta(0.3425): 0.153700%
theta(0.4164): 0.125659%
theta(0.4986): -0.041470%
theta(0.5808): 0.079302%
theta(0.6658): 0.125025%
theta(0.7479): 0.024419%
theta(0.8384): 0.071200%
theta(0.9178): 0.032841%
theta(1.0000): 0.030964%
theta(1.4986): -0.021227%
theta(2.0000): 0.147546%
theta(3.0055): 0.359198%
theta(4.0055): 0.542529%
theta(5.0027): 0.840753%
theta(6.0027): 0.932951%
theta(7.0027): 1.040001%
theta(8.0110): 1.159568%
theta(9.0082): 1.286390%
theta(10.0082): 1.251110%
theta(12.0082): 1.260508%
theta(15.0110): 1.306324%
theta(20.0164): 1.164882%
theta(25.0219): 1.118646%
theta(30.0192): 0.996521%
theta(40.0274): 0.535297%
theta(50.0329): 0.495002%


## Simulación

Se definen los parámetros requeridos, tanto del modelo como de la simulación.

In [45]:
num_sims = 1000
num_steps = 528 # 264 * 2 (2 años)

Se obtienen las trayectorias.

In [46]:
time_steps, paths = hw.sim_hw_many(gamma, sigma, theta, r0, 1000, 528, seed=1000)

**Ejercicio:** construir una función que retorne la función `sim_hw_many` pero sólo con los argumentos r0, num_sims, num_:steps y seed. O sea, que gamma, sigma y theta queden encerrados (closure) dentro de la función.

In [47]:
time_steps[0:20]

array([0.        , 0.00378788, 0.00757576, 0.01136364, 0.01515152,
       0.01893939, 0.02272727, 0.02651515, 0.03030303, 0.03409091,
       0.03787879, 0.04166667, 0.04545455, 0.04924242, 0.0530303 ,
       0.05681818, 0.06060606, 0.06439394, 0.06818182, 0.0719697 ])

In [13]:
paths[0][0:10] # son los primeros 10 valores de la primera trayectoria simulada.

array([0.0007891 , 0.00073774, 0.00082384, 0.00082725, 0.00091151,
       0.00083472, 0.00085602, 0.00079933, 0.00069104, 0.00076107])

Se extrae el elemento correspondiente a 6M, 1Y, 18M de cada simulación:

EE = Promedio(max(Vi, 0)) y se va a calcular en estos tiempos de parada o stopping times.

In [14]:
last_r = [(s[132], s[264], s[396]) for s in paths]
last_r[0]

(0.00023065735784508546, -0.0022857834272125696, -0.0010278696736560555)

## Cálculo de la *EE* de un Swap

Para el swap vamos a utilizar una función sencilla para la pata fija y consideraremos que el valor presente de la pata flotante siempre es igual al nocional. Consideremos un swap a 2Y.

In [15]:
swap_fija = inst.bono_tasa_fija(0, .5, 4, .001)

In [16]:
swap_fija

[(0.5, 0.05), (1.0, 0.05), (1.5, 0.05), (2.0, 100.05)]

Se define una función que construye una función que valoriza el swap a partir de $t$ y $r_t$.

In [17]:
def valorizador_swap(pata_fija, df_function):
    def valor(r, t):
        result = 0.0
        for e in pata_fija:
            if e[0] > t:
                result += e[1] * df_function(r, t, e[0])
        return result - 100.0 if result > 0.0 else 0.0
    return valor

Probemos:

In [18]:
vswap = valorizador_swap(swap_fija, hwz)

In [19]:
vswap(r0, 2)

0.0

Valoricemos el swap en 6M más en cada simulación:

In [27]:
valor_swap_6m = [vswap(r[0], .5) for r in last_r]
valor_swap_6m[0: 10]

[0.1087151035156495,
 -0.06891838127835115,
 -0.08828135334222509,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 -0.10416145596083481,
 -0.012453256721840944]

Mejor aún, podemos calcular directamente la exposición.

In [21]:
e_swap_6m = [max(vswap(r[0], .5), 0) for r in last_r]
e_swap_6m[0: 10]

[0.1087151035156495,
 0,
 0,
 0.024086600811841663,
 0.023152978606702845,
 0.16379864922694765,
 0.1809763078510258,
 0.14545651414299243,
 0,
 0]

Por lo tanto, la exposición esperada en 6M es:

In [22]:
ee_swap_6m = np.average(e_swap_6m)

In [23]:
print(f'La EE del swap en 6M es: {ee_swap_6m: ,.2f}')

La EE del swap en 6M es:  0.09


**Ejercicio:** completar el cálculo del CVA.

Se calcula la *EE* a 1Y y a 18M.

In [24]:
e_swap_1y = [max(vswap(r[1], 1), 0) for r in last_r]
e_swap_1y[0: 10]

[0.24063608214910914,
 0.17092867198768147,
 0.10516439489080653,
 0.07754376339669022,
 0,
 0.10064812920498412,
 0.040880910676904136,
 0.12907264180080347,
 0.016080984047064817,
 0]

In [25]:
e_swap_18m = [max(vswap(r[2], 1.5), 0) for r in last_r]
e_swap_18m[0: 10]

[0.08859937173058086,
 0.05476454386807461,
 0.10545540933068764,
 0.03618827563377636,
 0.02995438503567982,
 0.1415773018194102,
 0,
 0.09010082539154496,
 0.008504017457028112,
 0]

Luego, se calcula el factor de descuento a cada uno de los plazos en que se calcula *EE*: 6M, 1Y y 18M.

- Se extraen los resultados de la simulación hasta 6M, 1Y y 18M.

In [34]:
paths_up_to_6m = [s[0:132] for s in paths]
paths_up_to_1y = [s[0:264] for s in paths]
paths_up_to_18m = [s[0:396] for s in paths]

Luego, se calculan los factores de descuento "devolviéndose por cada trayectoria".

In [35]:
dt = 1 / 264.0

In [36]:
dfs_up_to_6m = [np.exp(-dt * np.sum(path)) for path in paths_up_to_6m]
dfs_up_to_1y = [np.exp(-dt * np.sum(path)) for path in paths_up_to_1y]
dfs_up_to_18m = [np.exp(-dt * np.sum(path)) for path in paths_up_to_18m]

In [44]:
pv_ee_6m = [both[0] * both[1] for both in zip(e_swap_6m, dfs_up_to_6m)]
pv_ee_1y = [both[0] * both[1] for both in zip(e_swap_1y, dfs_up_to_1y)]
pv_ee_18m = [both[0] * both[1] for both in zip(e_swap_18m, dfs_up_to_18m)]

In [47]:
mean_pv_ee_6m = np.mean(pv_ee_6m)
mean_pv_ee_1y = np.mean(pv_ee_1y)
mean_pv_ee_18m = np.mean(pv_ee_18m)
print(mean_pv_ee_6m, mean_pv_ee_1y, mean_pv_ee_18m)

0.09061139204024957 0.0791395253127055 0.049810373539262105
